# Exploring Hacker News Posts 

The dataset for this project is from Hacker News, a website started by Y Combinator.
The dataset was reduced from 300k rows to 20k rows, as submissions that didn't even receive comments were removed, and the remaining submissions were randomly sampled,

Dataset Dictionary:
- id: the unique identifier from Hacker News for the post
- title: the title of the post
- url: the URL that the posts links to, if the post has a URL
- num_points: the number of points the post acquired, calculated as the total number of - upvotes minus the total number of downvotes
- num_comments: the number of comments on the post
- author: the username of the person who submitted the post
- created_at: the date and time of the post's submission

For this project, we're intersted in posts with titles of either "Ask HN" or "Show HN"

We'll compare these two types of posts to determine the following:
- Do Ask HN or Show HN receive more comments on average?
- Do posts created at a certain time receive more comments on average?

# Removing Headers from List of Lists
When importing the dataset, we want to separate the row of headers vs the rows of the actual data

In [3]:
from csv import reader
opened_file = open('hacker_news.csv')
read_file = reader(opened_file)
hn = list(read_file)
headers = hn[0]
hn = hn[1:]

print(headers)
print(hn[0:5])




['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']
[['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01'], ['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']]


# Extracting Ask HN and Show HN Posts
We'll begin the analysis by creating 3 lists: one for ask posts, one for show posts, and other.

In [4]:
#Use string method .startswith()
#example: print('dataquest'.startswith('Data')) = False

ask_posts = []
show_posts = []
other_posts = []

for row in hn:
    title = row[1]
    if title.lower().startswith('ask hn'): #by making string lower, inclusive of any upper cases
        ask_posts.append(row)
    elif title.lower().startswith('show hn'):
        show_posts.append(row)
    else:
        other_posts.append(row)
            
#Count the number of posts in each list 
#Total = 20.1k
print('Ask Posts:', len(ask_posts))
print('Show Posts:', len(show_posts))
print('Other Posts:',len(other_posts))

Ask Posts: 1744
Show Posts: 1162
Other Posts: 17194


In [5]:
#First 5 rows from the Ask Post List
print(ask_posts[0:5])

#First 5 rows from the Show Post List
print(show_posts[0:5])



[['12296411', 'Ask HN: How to improve my personal website?', '', '2', '6', 'ahmedbaracat', '8/16/2016 9:55'], ['10610020', 'Ask HN: Am I the only one outraged by Twitter shutting down share counts?', '', '28', '29', 'tkfx', '11/22/2015 13:43'], ['11610310', 'Ask HN: Aby recent changes to CSS that broke mobile?', '', '1', '1', 'polskibus', '5/2/2016 10:14'], ['12210105', 'Ask HN: Looking for Employee #3 How do I do it?', '', '1', '3', 'sph130', '8/2/2016 14:20'], ['10394168', 'Ask HN: Someone offered to buy my browser extension from me. What now?', '', '28', '17', 'roykolak', '10/15/2015 16:38']]
[['10627194', 'Show HN: Wio Link  ESP8266 Based Web of Things Hardware Development Platform', 'https://iot.seeed.cc', '26', '22', 'kfihihc', '11/25/2015 14:03'], ['10646440', 'Show HN: Something pointless I made', 'http://dn.ht/picklecat/', '747', '102', 'dhotson', '11/29/2015 22:46'], ['11590768', 'Show HN: Shanhu.io, a programming playground powered by e8vm', 'https://shanhu.io', '1', '1', 'h

# Calculating the Average Number of Comments for Ask HN and Show HN Posts

In [6]:
# Find total number of comments in ask posts

total_ask_comments = 0

# In the header, item 4 is the Comments column
for row in ask_posts:
    num_com = int(row[4])
    total_ask_comments += num_com
    
print('Total Ask Comments', total_ask_comments)
num_ask_post = len(ask_posts)
print('Total Ask Posts', num_ask_post)
avg_ask_comments = round((total_ask_comments / num_ask_post), 0)
print('Average Comments Per Post:', avg_ask_comments)


Total Ask Comments 24483
Total Ask Posts 1744
Average Comments Per Post: 14.0


In [7]:
#Find total number of comments in show posts

total_show_comments = 0

for row in show_posts:
    num_com = int(row[4])
    total_show_comments += num_com
    
print('Total Show Comments:', total_show_comments)
num_show_post = len(show_posts)
print('Total Show Posts:',num_show_post)
avg_show_comments = round(total_show_comments / num_show_post, 0)
print('Average Show Comments Per Post:',avg_show_comments)


Total Show Comments: 11988
Total Show Posts: 1162
Average Show Comments Per Post: 10.0


From the above calculations, ASK POSTS get an average of 14 comments per post compared to SHOW POSTS get an average of 10 comments per post. 

# Finding the Number of Ask Posts and Comments by Hour Created

We'll determine if ask posts created at a certain time are more likely to attract comments. We'll use the following steps to perform this analysis:

- Calculate the number of ask posts created in each hour of the day, along with the number of comments received.
- Calculate the average number of comments ask posts receive by hour created.

In [8]:
import datetime as dt

result_list = [] # list of lists 1st element is create date, 2nd element is # of comments 

for row in ask_posts:
    create_dte = row[6]
    num_comment = int(row[4])
    new_list = [create_dte, num_comment]
    result_list.append(new_list)

#print(result_list)
counts_by_hour = {} #aka posts per hour
comments_by_hour = {}

for row in result_list:
    date_1_str = row[0]
    date_1_dt_strp = dt.datetime.strptime(date_1_str, "%m/%d/%Y %H:%M")
    #print(date_1_dt_strp)
    date_1_dt_strft = date_1_dt_strp.strftime( "%H")
    #print(date_1_dt_strft)
    if date_1_dt_strft not in counts_by_hour:
        counts_by_hour[date_1_dt_strft] = 1
        comments_by_hour[date_1_dt_strft] = row[1]
    elif date_1_dt_strft in counts_by_hour:
        counts_by_hour[date_1_dt_strft] += 1
        comments_by_hour[date_1_dt_strft] += row[1]

    
print(counts_by_hour)
print('\n')
print(comments_by_hour)
    

    
    
    
    

{'09': 45, '13': 85, '10': 59, '14': 107, '16': 108, '23': 68, '12': 73, '17': 100, '15': 116, '21': 109, '20': 80, '02': 58, '18': 109, '03': 54, '05': 46, '19': 110, '01': 60, '22': 71, '08': 48, '04': 47, '00': 55, '06': 44, '07': 34, '11': 58}


{'09': 251, '13': 1253, '10': 793, '14': 1416, '16': 1814, '23': 543, '12': 687, '17': 1146, '15': 4477, '21': 1745, '20': 1722, '02': 1381, '18': 1439, '03': 421, '05': 464, '19': 1188, '01': 683, '22': 479, '08': 492, '04': 337, '00': 447, '06': 397, '07': 267, '11': 641}


# Calculating Average Number of Comments for Ask HN Posts by Hour

The counts_by_hour dictionary contains the number of ask posts created during each hour of the day.

The comments_by_hour dictionary contains the corresponding number of comments ask posts created at each hour received.

We'll use these two dictionaries to calculate the average number of comments for posts created during each hour of the day.

In [9]:
avg_by_hour = []

for hour in comments_by_hour: #row = key
    avg_by_hour.append([hour, comments_by_hour[hour]/ counts_by_hour[hour]])

#First element is hour, 2nd element is num comments / num posts 
print(avg_by_hour)    
    

[['09', 5.5777777777777775], ['13', 14.741176470588234], ['10', 13.440677966101696], ['14', 13.233644859813085], ['16', 16.796296296296298], ['23', 7.985294117647059], ['12', 9.41095890410959], ['17', 11.46], ['15', 38.5948275862069], ['21', 16.009174311926607], ['20', 21.525], ['02', 23.810344827586206], ['18', 13.20183486238532], ['03', 7.796296296296297], ['05', 10.08695652173913], ['19', 10.8], ['01', 11.383333333333333], ['22', 6.746478873239437], ['08', 10.25], ['04', 7.170212765957447], ['00', 8.127272727272727], ['06', 9.022727272727273], ['07', 7.852941176470588], ['11', 11.051724137931034]]


# Sorting the values from a list of lists

Sort the list of lists and print 5 highest values in an easy to read manner

In [12]:
swap_by_avg_hour = []

for row in avg_by_hour:
    swap_by_avg_hour.append([row[1], row[0]])

print(swap_by_avg_hour)

print('\n')

sorted_swap = sorted(swap_by_avg_hour, reverse = True)
#print(sorted_swap)

print("Top 5 Hours for Ask Posts Comments")

for row in sorted_swap[0:5]:
    hour = row[1]
    date_1_dt_strp = dt.datetime.strptime(hour, "%H")
    date_1_dt_strft = date_1_dt_strp.strftime( "%H:%M")
    print("{}: {:.2f} average comments per post".format(date_1_dt_strft, row[0]))

    #print(date_1_dt_strp)
    #print(date_1_dt_strft)
        

[[5.5777777777777775, '09'], [14.741176470588234, '13'], [13.440677966101696, '10'], [13.233644859813085, '14'], [16.796296296296298, '16'], [7.985294117647059, '23'], [9.41095890410959, '12'], [11.46, '17'], [38.5948275862069, '15'], [16.009174311926607, '21'], [21.525, '20'], [23.810344827586206, '02'], [13.20183486238532, '18'], [7.796296296296297, '03'], [10.08695652173913, '05'], [10.8, '19'], [11.383333333333333, '01'], [6.746478873239437, '22'], [10.25, '08'], [7.170212765957447, '04'], [8.127272727272727, '00'], [9.022727272727273, '06'], [7.852941176470588, '07'], [11.051724137931034, '11']]


Top 5 Hours for Ask Posts Comments
15:00: 38.59 average comments per post
02:00: 23.81 average comments per post
20:00: 21.52 average comments per post
16:00: 16.80 average comments per post
21:00: 16.01 average comments per post


The above times are in EST. Based on the results, posts posted on 3PM EST on average have the highest amount of comments. In terms of the times, all 5 results are after 12PM EST. 8PM and 9PM would be times where people are going chilling for the night and has time to comment on posts. 

# Conclusion

The analysis aimed to gain insights into 2 questions:
- Do Ask HN or Show HN receive more comments on average?
- Do posts created at a certain time receive more comments on average?

For question one, we determined that Ask Posts on average received more comments than Show Posts.

For question two, we determined that the time where ask posts received the most comments on average was 3PM EST. The top 5 values all fall after 12PM EST
